**`Compare runs; using hyperparameter tuning, Choose a model & Deploy it to a REST API`**

In this section, the following will be carried out;

- Run a hyperparameter sweep using `hyperopt`.
- Compare results of the runs in the MLflow UI.
- Choose the best run and register it as a model.
- Deploy the model to a REST API
- Build a container image suitable for deployment to a cloud environment.

In [1]:
import warnings
warnings.filterwarnings("ignore", category = DeprecationWarning)

import keras
import numpy as np
import pandas as pd
import mlflow

from mlflow.models import infer_signature
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


In [2]:
""" RUN THE COMMAND BELOW IN THE TERMINAL. """
# export MLFOW_TRACKING_URI=http://localhost:5000

mlflow.set_tracking_uri(uri = "http://127.0.0.1:8080")

Now load the Dataset and split into it into training, validation and test sets.

In [3]:
data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep = ";"
)

In [4]:
train, test = train_test_split(data, test_size = 0.25, random_state = 42)
train_X = train.drop(["quality"], axis = 1).values
train_y = train[["quality"]].values.ravel()
test_X = test.drop(["quality"], axis = 1).values
test_y = test[["quality"]].values.ravel()

train_X, validation_X, train_y, validation_y = train_test_split(
    train_X, train_y, test_size = 0.2, random_state = 42
)

In [5]:
# Infer madel signature.

signature = infer_signature(train_X, train_y)

Define the Model Architecture.


The model uses MLFlow to track the `parameters`, `results`, and the `model` itself of each trail as a child run.

In [6]:
def model_train(params, epochs, train_X, train_y, validation_X, validation_y, test_X, test_y):
    mean = np.mean(train_X, axis = 0)
    var = np.var(train_X, axis = 0)
    
    model = keras.Sequential(
        [
            keras.Input([train_X.shape[1]]),
            keras.layers.Normalization(mean = mean, variance = var),
            keras.layers.Dense(64, activation = "relu"),
            keras.layers.Dense(1)
        ]
    )
    
    # Compile model
    model.compile(
        optimizer = keras.optimizers.SGD(
            learning_rate = params["lr"],
            momentum = params["momentum"]
        ),
        loss = "mean_squared_error",
        metrics = [keras.metrics.RootMeanSquaredError()]
    )
    
    # Train the model MLflow tracking
    
    with mlflow.start_run(nested = True):
        model.fit(
            train_X,
            train_y,
            validation_data = (validation_X, validation_y),
            epochs = epochs,
            batch_size = 64
        )
        
        # Evaluate the model
        eval_result = model.evaluate(validation_X, validation_y, batch_size = 64)
        eval_rmse = eval_result[1]
        
        # Log parameters and results
        mlflow.log_params(params = params)
        mlflow.log_metric("eval_rmse", eval_rmse)
        
        # Log model.
        mlflow.tensorflow.log_model(model, "model", signature = signature)
        
        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

Define an Objective function.

This function takes the `hyperparameters` and returns the `results` of the **model_train** function for that set of `hyperparameters`.

In [7]:
def objective(params):
    # MLFlow will track the hyperparameters & results for each run.
    
    result = model_train(
        params = params,
        epochs = 3,
        train_X = train_X,
        train_y = train_y,
        validation_X = validation_X,
        validation_y = validation_y,
        test_X = test_X,
        test_y = test_y
    )
    return result

Define the Search Space.

Next, we will define the search space for `Hyperopt`. In this case, we want to try different values of `learning-rate` and `momentum`. Hyperopt begins its optimization process by selecting an initial set of `hyperparameters`, typically chosen at random or based on a specified domain space. This domain space defines the range and distribution of possible values for each hyperparameter. After evaluating the initial set, Hyperopt uses the results to update its `probabilistic model`, guiding the selection of subsequent hyperparameter sets in a more informed manner, aiming to converge towards the optimal solution.

In [8]:
space = {
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0, 1.0)
}

Set MLflow Experiments & runs.

Finally, we will run the `hyperparameter` sweep using `Hyperopt`, passing in the `objective function` and `search space`. Hyperopt will try different hyperparameter combinations and return the results of the best one. We will store the best parameters, model, and evaluation metrics in `MLflow`.

In [9]:
mlflow.set_experiment("/wine-quality")

with mlflow.start_run():
    # Conduct the hp search
    trials = Trials()
    best = fmin(
        fn = objective,
        space = space,
        algo = tpe.suggest,
        max_evals = 12,
        trials = trials
    )
    
    # Fetch the details of the best run
    best_run = sorted(trials.results, key = lambda x: x["loss"])[0]
    
    # Log the best parameters, loss, and model.
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature = signature)
    
    # Print the best parameters and coresponding loss.
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")

  0%|          | 0/12 [00:00<?, ?trial/s, best loss=?]WARNING:tensorflow:From c:\Users\DELL\anaconda3\Lib\site-packages\keras\src\backend.py:1398: The name tf.executing_eagerly_outside_functions is deprecated. Please use tf.compat.v1.executing_eagerly_outside_functions instead.

Epoch 1/3                                             

  0%|          | 0/12 [00:00<?, ?trial/s, best loss=?]WARNING:tensorflow:From c:\Users\DELL\anaconda3\Lib\site-packages\keras\src\utils\tf_utils.py:492: The name tf.ragged.RaggedTensorValue is deprecated. Please use tf.compat.v1.ragged.RaggedTensorValue instead.

46/46 [==============================] - 1s 7ms/step - loss: 3.3792 - root_mean_squared_error: 1.8383 - val_loss: 1.2473 - val_root_mean_squared_error: 1.1168

Epoch 2/3                                             

46/46 [==============================] - 0s 3ms/step - loss: 0.9655 - root_mean_squared_error: 0.9826 - val_loss: 0.8338 - val_root_mean_squared_error: 0.9132

Epoch 3/3               

INFO:tensorflow:Assets written to: C:\Users\DELL\AppData\Local\Temp\tmppgx53zgr\model\data\model\assets

c:\Users\DELL\anaconda3\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")



Epoch 1/3                                                                       

46/46 [==============================] - 1s 6ms/step - loss: 4.6531 - root_mean_squared_error: 2.1571 - val_loss: 1.3548 - val_root_mean_squared_error: 1.1640

Epoch 2/3                                                                       

46/46 [==============================] - 0s 3ms/step - loss: 1.0915 - root_mean_squared_error: 1.0447 - val_loss: 1.0068 - val_root_mean_squared_error: 1.0034

Epoch 3/3                                                                       

46/46 [==============================] - 0s 2ms/step - loss: 0.8456 - root_mean_squared_error: 0.9196 - val_loss: 0.7988 - val_root_mean_squared_error: 0.8937

12/12 [==============================] - 0s 2ms/step - loss: 0.7988 - root_mean_squared_error: 0.8937

  8%|▊         | 1/12 [00:19<03:17, 17.95s/trial, best loss: 0.8185364603996277]INFO:tensorflow:Assets written to: C:\Users\DELL\AppData\Local\Temp\tmpzgjcn84j\model\data\

INFO:tensorflow:Assets written to: C:\Users\DELL\AppData\Local\Temp\tmpzgjcn84j\model\data\model\assets



Epoch 1/3                                                                       

46/46 [==============================] - 1s 7ms/step - loss: 30.5049 - root_mean_squared_error: 5.5231 - val_loss: 29.8034 - val_root_mean_squared_error: 5.4592

Epoch 2/3                                                                       

46/46 [==============================] - 0s 2ms/step - loss: 29.2593 - root_mean_squared_error: 5.4092 - val_loss: 28.5909 - val_root_mean_squared_error: 5.3470

Epoch 3/3                                                                       

46/46 [==============================] - 0s 2ms/step - loss: 28.0660 - root_mean_squared_error: 5.2977 - val_loss: 27.4283 - val_root_mean_squared_error: 5.2372

12/12 [==============================] - 0s 2ms/step - loss: 27.4283 - root_mean_squared_error: 5.2372

 17%|█▋        | 2/12 [00:33<02:34, 15.47s/trial, best loss: 0.8185364603996277]INFO:tensorflow:Assets written to: C:\Users\DELL\AppData\Local\Temp\tmpuxpjglbl\mode

...

Epoch 1/3                                                                        

46/46 [==============================] - 1s 7ms/step - loss: 27.4487 - root_mean_squared_error: 5.2392 - val_loss: 23.3783 - val_root_mean_squared_error: 4.8351

Epoch 2/3                                                                        

46/46 [==============================] - 0s 3ms/step - loss: 19.7356 - root_mean_squared_error: 4.4425 - val_loss: 16.8939 - val_root_mean_squared_error: 4.1102

Epoch 3/3                                                                        

46/46 [==============================] - 0s 2ms/step - loss: 14.2598 - root_mean_squared_error: 3.7762 - val_loss: 12.2657 - val_root_mean_squared_error: 3.5022

12/12 [==============================] - 0s 2ms/step - loss: 12.2657 - root_mean_squared_error: 3.5022

 92%|█████████▏| 11/12 [02:42<00:14, 14.40s/trial, best loss: 0.7167361974716187]INFO:tensorflow:Assets written to: C:\Users\DELL\AppData\Local\Temp\tmpx4szoz37\

INFO:tensorflow:Assets written to: C:\Users\DELL\AppData\Local\Temp\tmpx4szoz37\model\data\model\assets



100%|██████████| 12/12 [02:55<00:00, 14.64s/trial, best loss: 0.7167361974716187]
INFO:tensorflow:Assets written to: C:\Users\DELL\AppData\Local\Temp\tmpy1yt71wq\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\DELL\AppData\Local\Temp\tmpy1yt71wq\model\data\model\assets



Best parameters: {'lr': 0.06684123135810843, 'momentum': 0.567042058031489}
Best eval rmse: 0.7167361974716187
